In [0]:
import os
os.environ['DISABLE_COLAB_TF_IMPORT_HOOK'] = '1'

In [3]:
from __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import os
import time
import keras

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path_to_file='/content/drive/My Drive/SpongeBobDataset.txt'

In [6]:
with open(path_to_file,'rb') as f:
  text=f.read().decode(encoding='utf-8')
  '''NOTE: The .txt file which was initially uploaded was throwing
  an error as Python could not encode it. However, replacing the file with
  a pre-encoded file (in utf-8) fixed it'''
#text=text.rstrip('\n').decode('utf-8')
#text=text.split('\r\n')
print('Length of text: {} charachters'.format(len(text)))

Length of text: 649746 charachters


In [7]:
print(text[:250])

﻿[Following the SpongeBob SquarePants theme song, the brief opening titles show the names of creator Stephen Hillenburg, Derek Drymon and others. The episode opens with a bubble transition, and we see a coral reef under the sea. The camera zooms to i


In [8]:
#Unique characters
vocab=sorted(set(text))
print('{} unique characters'.format(len(vocab)))

102 unique characters


In [0]:
#Vectorization of text
char2idx={u:i for i,u in enumerate(vocab)}
idx2char=np.array(vocab)

text_as_int=np.array([char2idx[c] for c in text])

In [10]:
print('{')
for char,_ in zip(char2idx,range(20)):
  print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))
print(' ...\n}')

print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

{
 '\t':   0,
 '\n':   1,
 '\r':   2,
 ' ' :   3,
 '!' :   4,
 '"' :   5,
 '#' :   6,
 '$' :   7,
 '%' :   8,
 '&' :   9,
 "'" :  10,
 '(' :  11,
 ')' :  12,
 '*' :  13,
 ',' :  14,
 '-' :  15,
 '.' :  16,
 '/' :  17,
 '0' :  18,
 '1' :  19,
 ...
}
'\ufeff[Following t' ---- characters mapped to int ---- > [101  56  36  72  69  69  72  80  66  71  64   3  77]


In [11]:
#Max length sentence we want for a single input in characters
seq_length=150
examples_per_epoch=len(text)//seq_length

#create training examples and targets
char_dataset=tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

﻿
[
F
o
l


In [12]:
sequences=char_dataset.batch(seq_length+1,drop_remainder=True)
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\ufeff[Following the SpongeBob SquarePants theme song, the brief opening titles show the names of creator Stephen Hillenburg, Derek Drymon and others. The e'
'pisode opens with a bubble transition, and we see a coral reef under the sea. The camera zooms to initiate parallax scrolling, which reveals the city o'
'f Bikini Bottom. It continues zooming to show a brown rock, a Moai head, and a pineapple, which each contain inhabitants.]\r\nFrench Narrator: Ah, the se'
"a... so fascinating. So wonderful. Here, we see Bikini Bottom, teeming with life. [Shows from left to right Patrick's, Squidward's, and SpongeBob's hou"
"ses. Zooms in on SpongeBob's house.] Home to one of my favorite creatures, SpongeBob SquarePants. Yes, of course he lives in a pineapple, you silly. [S"


In [13]:
def split_input_target(chunk):
  input_text=chunk[:-1]
  target_text=chunk[1:]
  return input_text,target_text
dataset=sequences.map(split_input_target)

for input_example,target_example in dataset.take(1):
  print('Input data: ',repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ',repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\ufeff[Following the SpongeBob SquarePants theme song, the brief opening titles show the names of creator Stephen Hillenburg, Derek Drymon and others. The '
Target data:  '[Following the SpongeBob SquarePants theme song, the brief opening titles show the names of creator Stephen Hillenburg, Derek Drymon and others. The e'


In [14]:
#Prediction of next character
for i, (input_idx,target_idx) in enumerate(zip(input_example[:13],target_example[:13])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 101 ('\ufeff')
  expected output: 56 ('[')
Step    1
  input: 56 ('[')
  expected output: 36 ('F')
Step    2
  input: 36 ('F')
  expected output: 72 ('o')
Step    3
  input: 72 ('o')
  expected output: 69 ('l')
Step    4
  input: 69 ('l')
  expected output: 69 ('l')
Step    5
  input: 69 ('l')
  expected output: 72 ('o')
Step    6
  input: 72 ('o')
  expected output: 80 ('w')
Step    7
  input: 80 ('w')
  expected output: 66 ('i')
Step    8
  input: 66 ('i')
  expected output: 71 ('n')
Step    9
  input: 71 ('n')
  expected output: 64 ('g')
Step   10
  input: 64 ('g')
  expected output: 3 (' ')
Step   11
  input: 3 (' ')
  expected output: 77 ('t')
Step   12
  input: 77 ('t')
  expected output: 65 ('h')


In [15]:
#Batch Size
batch_size=64
steps_per_epoch=examples_per_epoch/batch_size

#buffer size to shuffle the dataset
buffer_size=10000

dataset=dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)
print(dataset)

<DatasetV1Adapter shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


In [0]:
#length of vocabulary in chars
vocab_size=len(vocab)
#embedding dimensions
embedding_dim=256
#no. of rnn units (GRU) in this case
rnn_units=1024

In [0]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

#Taken from tensoflow docs as there was some problem in intializing a CuDNN
#layer with a CUDA GPU on Colab. The rnn would probably be the one under the
#[else:] block

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=batch_size)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 150, 102) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           26112     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 102)           104550    
Total params: 4,065,894
Trainable params: 4,065,894
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices=tf.random.categorical(example_batch_predictions[0],num_samples=1)
sampled_indices=tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
sampled_indices 

array([  4,  71,  68,  95,  97,  92,  91, 101,  37,  28,  41,  74,   2,
         2,  83,  51,  33,  54,  51,  64,  93,  25,  44,  85,  87,  63,
        46,  81,  54,  27,  34,  26,  75,  16,  13,  67,  26,  96,  38,
        52,  95,  35,  19,  37,  43,  15,  78,  92,  29,  54,  30,  61,
        90,  48,   5,  39,  71,  91,  91,  83,  39,  17,  87,  73,  79,
        81,  22,  42,  30,  40,  60,  57,  64,  56,  70,  61,  31,  36,
        25,  73,  35,  56,  70,  11,  92,  13, 100,  79,  82,  26,  66,
        82,  40,  60,  51,  78,  57,  10,   9,  76,  44,  39,  71,  78,
        80,  88,  40, 100,  37,  94,  80,  62,  11,  47, 101,  42,  24,
        78,  84,  96,  37,  91,  65,  10,  69,  76,  56,  76,  55,  36,
         3,  40,  38,  93,  12,  14,  40,  25,  12,  17,  93,  31,  98,
        87,   4,  17,  32,   3,  69, 100])

In [24]:
#sample predictions from untrained model
print('Input: \n',repr(''.join(idx2char[input_example_batch[0]])))
print()
print('Next Char Predictions: \n',repr(''.join(idx2char[sampled_indices ])))

Input: 
 "d his name is...SpongeBob SquarePants! [echoes] SpongeBob SquarePants! SpongeBob SquarePants! [fades to a vision of Mrs. Puff's fears] La La La La! [h"

Next Char Predictions: 
 '!nk♪뒤“’\ufeffG:Kq\r\rzUCYUg”7N¿ïfPxY9D8r.*j8가HV♪E1GM-u“;Y?d‘R"In’’zI/ïpvx4L?Jc]g[mdAF7pE[m(“*으vy8iyJcUu]\'&sNInuw–J으G…we(Q\ufeffL6u¢가G’h\'ls[sZF JH”),J7)/”A로ï!/B l으'


In [25]:
def loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

example_batch_loss=loss(target_example_batch,example_batch_predictions)
print('Prediction Shape: ',example_batch_predictions.shape)
print('Scalar Loss: ',example_batch_loss.numpy().mean())

Prediction Shape:  (64, 150, 102)
Scalar Loss:  4.6257887


In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(),loss=loss)

In [0]:
#Configuring Checkpoints
checkpoint_dir='./training_checkpoints'
#Name of the checkpoint files
checkpoint_prefix=os.path.join(checkpoint_dir,'ckpt_{epoch}')

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
filepath=checkpoint_prefix,
save_weights_only=True)

In [28]:
epochs=15
history=model.fit(dataset.repeat(),epochs=epochs,steps_per_epoch=steps_per_epoch,
                 callbacks=[checkpoint_callback])

Epoch 1/15
68/67 [==============================] - 1069s 16s/step - loss: 3.3859
Epoch 2/15
68/67 [==============================] - 994s 15s/step - loss: 2.2316
Epoch 3/15
68/67 [==============================] - 1047s 15s/step - loss: 1.9533
Epoch 4/15
68/67 [==============================] - 1026s 15s/step - loss: 1.7883
Epoch 5/15
68/67 [==============================] - 983s 14s/step - loss: 1.6614
Epoch 6/15
68/67 [==============================] - 1009s 15s/step - loss: 1.5497
Epoch 7/15
68/67 [==============================] - 1025s 15s/step - loss: 1.4587
Epoch 8/15
68/67 [==============================] - 1012s 15s/step - loss: 1.3819
Epoch 9/15
68/67 [==============================] - 1003s 15s/step - loss: 1.3183
Epoch 10/15
68/67 [==============================] - 1024s 15s/step - loss: 1.2637
Epoch 11/15
68/67 [==============================] - 1004s 15s/step - loss: 1.2187
Epoch 12/15
68/67 [==============================] - 1003s 15s/step - loss: 1.1802
Epoch 13/15
68/

In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_15'

In [0]:
model=build_model(vocab_size,embedding_dim,rnn_units,batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            26112     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 102)            104550    
Total params: 4,065,894
Trainable params: 4,065,894
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Taken from tensorflow rnn docs
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 10000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [35]:
print(generate_text(model,start_string=u"SpongeBob: "))

SpongeBob: [he pulls a button on the floor] Much man leftbing on the bubble! Do you think yes?!
SpongeBob: Uh, what's that.
Patrick: What's no one: Texas saud shase we can come need feel you bo this bike!
Plankton: Polo. How did you're heard of chashes?
Patrick: Yeah!
Squidward: [comes up and dall] from my boy and pet you were a Bike!
Patrick: Get out on that pizza! [Holds up a rock snack] Beware the one? [SpongeBob insides read] Dah-23! Hell's memberiver evsleamer.
[Mr. Krabs tries all the horaps from him. Patrick warks to the kitting your complete safily. He didn't know how to tie that, scary thought becoming missing mator cash togelity.
Bubble Bass: There's only like things from Sandy. [he pulls laughing]
Pearl: Don't was just happened?
Putty: Whoaing through it, rings!
Old: Darn it. Let me sigh $5:0000 well get my signy. [nozing mink, I in that plane.
SpongeBob: Patrick! Why don't leaving it. Get Giant Krabby Patty. Perch Perkeds the screen] Those are knows! [walks outside. Lands o

As we cann see, the output generated is barely comprehensible. However, we need to keep in mind that this is a character-based RNN, ie. that when the training started, it did not know what words were (see the predictions from untrained model).

However, to (my) satisfaction, the RNN has atleast been able to get most of the names correct and in the format of the transcript (square-brackets showing feelings, etc).

The model took 4.5 hours to train on a Google Colab GPU (without acceleration). 
For more accurate results, the easiest way would be to increase the number of epochs (30, 50...) or mess with the temperature setting